## Getting data from the Web

### The Internet and HTTP

The internet (you may have heard of it) is a system of computer networks that uses internet protocols to link communicating devices.
The internet is dictated by network protocols, which are rules by which computers communicate.
Each protocol is designed for a certain task and is specific to a type of data, physical devices, or application.
So for example, the Simple Mail Transfer Protocol (SMTP) and Post Office Protocol (POP), are agreements on how email clients and servers create and parse messages.
These protocols are mostly open source, such as HTTP, but some are proprietary, like the Yahoo! Instant Messenger Protocol.
  The cryptocurrency, Bitcoin, has an associated protocol that specifies how bitcoins are sent and recieved.
  The internet protocol suite is a stack of interdependent protocols that power the internet, allowing for the routing of packets (IP), the interaction with physical components (MAC/Ethernet/etc.), the error-free transmission of data (TCP), and the application layer that standardizes communication (HTTP).
  You can see a list of network protocols at <a href="https://en.wikipedia.org/wiki/Lists_of_network_protocols">this wikipedia page</a>.

Bob Kahn and Vinton Cerf developed TCP/IP for DARPA, the research arm of the Department of Defense.
The computer network at the time was called ARPANET, and the protocols multiplied as TCP spawned IP, and other researchers joined the project.
Later Tim Berners-Lee at the European Organization for Nuclear Research (CERN) developed the Hypertext Transfer Protocol (HTTP).
HTTP is the application protocol that is the basis for the world wide web.
We will discuss more about HTML, the hypertext markup language in another lesson, but for now, just know that it allows for nodes to be linked to one another (via URIs) and these nodes have structured text.
HTTP standards are maintained and updated by the Internet Engineering Task Force (IETF) and the World Wide Web Consortium (W3C).
A very interesting account of the development and implications of the internet can be found in the documentary, "Lo and Behold, Reveries of the Connected World" by Werner Herzog (available on Netflix as of Feb 2017).

Network protocols that are maintained by the IETF are published in Request for Comment (RFC) documents.
For example, we can see the first RFC for HTTP (<a href="https://tools.ietf.org/html/rfc1945">RFC 1945</a>) has the following abstract.
<center><pre>Network Working Group                                     T. Berners-Lee
Request for Comments: 1945                                       MIT/LCS
Category: Informational                                      R. Fielding
                                                               UC Irvine
                                                              H. Frystyk
                                                                 MIT/LCS
                                                                May 1996


Hypertext Transfer Protocol -- HTTP/1.0
    
Abstract

   The Hypertext Transfer Protocol (HTTP) is an application-level
   protocol with the lightness and speed necessary for distributed,
   collaborative, hypermedia information systems. It is a generic,
   stateless, object-oriented protocol which can be used for many tasks,
   such as name servers and distributed object management systems,
   through extension of its request methods (commands). A feature of
   HTTP is the typing of data representation, allowing systems to be
   built independently of the data being transferred.
</pre></center>

HTTP is based on the client-server computing model, where a client, typically a web browser, and an application on a web server, communicate to serve web content.
  One common open-source web server application is Apache (which is also the name of its parent organization that later developed the web browser Firefox).
  HTTP is a request-response protocol, meaning that clients make requests for content and then the server makes a response.
  There are several request methods to HTTP, notably GET which is used to retrieve data, and POST which asks the server to accept data at a URI.  


The Request package is a main tool making requests to web servers.  Because we will focus on extracting data from the web, we will focus on GET requests, and ignore POST requests.  The basic GET request is what the browser does when you navigate to a URL.  As it turns out you can pass parameters to URLs as well.  The URLs follow a certain format according to other RFCs that specify how URLs should be written.  The basic idea is that the URL can be broken into the following components: scheme, network location, hierarchical path, parameters, query, and fragment identifier.  It follows the basic format: <code>scheme://netloc/path;parameters?query#fragment</code>. The scheme can be <code>file, ftp, http, https,...</code>, the netloc is typically the host root url, like <code>www.google.com</code>, the path is a typical relative path, then the parameters and query specify arguments for GET, POST, PUT, etc.  So if we take the url <code>http://api.petfinder.com/my.method?key=12345&arg1=foo&token=67890&sig=abcdef</code> then it has scheme <code>http</code>, netloc of <code>api.petfinder.com</code>, path of <code>my.method</code>, and query is <code>key=12345&arg1=foo&token=67890&sig=abcdef</code>.  The functions <code>urlparse, urlunparse</code> will let you go from a url to a iterable of these components and back again.  You can find the [Request quickstart documentation here](http://docs.python-requests.org/en/master/user/quickstart/).

The basic use of the Requests package is through the GET method, as in the following.

In [7]:
import requests

r = requests.get('https://api.github.com/events')

This will establish a connection to github.com and then will make the get request.  This is the same as what your browser does when you type https://api.github.com/events into the url (go ahead and try it).  When we run this we are getting a summary of the public events in github through their web API, which we will discuss shortly.

We can see the text of what was returned from github by accessing the ``text`` attribute.  This is what the browser receives and needs to parse to display something meaningful to you.

In [8]:
r.text[:300]

'[{"id":"8530767475","type":"PushEvent","actor":{"id":324298,"login":"jimkang","display_login":"jimkang","gravatar_id":"","url":"https://api.github.com/users/jimkang","avatar_url":"https://avatars.githubusercontent.com/u/324298?"},"repo":{"id":129019986,"name":"jimkang/self-tagging-bot","url":"https:'

We are seeing a serialization of JSON data from a web API, which we will discuss shortly.

**Note:** You can use requests to make an overwhelmingly large number of requests to a webserver in a very short period of time.  This is how denial of service attacks work, and it is an unkind thing to do.  Please use requests with care.

## Web APIs and JSON

An application programming interface (API) is a set of methods by which software components communicate.
Web APIs define how HTTP request methods can be used to access and modify data on the server.
In modern web APIs you specify the URL of the API, such as <code>http://api.petfinder.com/subsystem.method</code> indicating the method, then specify arguments, and the desired data format (JSON, XML, etc.).
The API will read the parameters in the url, and then return raw JSON.

JSON, which is also specified by a request for comment (RFC 7159), is a simple data exchange format.  It is designed to efficiently parse complex data, but is not meant to be especially human readible and writtable like markup languages.  JSON is perfect for web APIs because they are meant to serve data to other applications and the occasional data scientist, and is not meant to be read as a webpage in your browser.

You should think of JSON in the same way that you think of CSV, it determines a way to turn a python object into a string that can be sent via HTTP or written to a file.  This conversion is called serialization, and to convert back is called deserialization.  JSON is very flexible since it is based on dictionaries and lists (called objects and arrays in JSON), and you can nest these in complex ways.  So for example, a matrix could be written as a list of lists, a DataFrame is a dictionary with column names for keys and lists for values, etc.  You can see a detailed description of JSON at <a href="http://json.org">json.org</a>.

We can access the parsed json from the request with the ``json`` method, as in,

In [12]:
gh_json = r.json()

Because serialized JSON creates arrays by encapsulating the array with square brackets [] and items separated by commas.  The object (like the Python dictionary) is encapsulated in curly brackets {} and key value pairs are separated by colons.  All of these match the way that we create lists and dictionaries in Python.  Critically, along with strings and numerical types the array and object types can be used for the values in the array and object, meaning that they can be nested.  We saw that the response json starts with ``[{"id":"8530767475",`` which means that the json has an array at the outer layer and then the first value is an object.  We can see this first fact by looking at the type of the parsed json.

In [13]:
type(gh_json)

list

Arrays are interpretted as lists, and similarly, the object is interpretted as a dictionary.

In [14]:
type(gh_json[0])

dict

So for example, we could get the id of the first entry of the response with,

In [15]:
gh_json[0]['id']

'8530767475'

Unfortunately, there is no standard format for the response of web APIs, except that they have to follow the grammar of json (or xml).  This means that reading the data from each will differ from API to API.

### Petfinder API

We found our dog Emmy (a mixed breed herding dog named after the German mathematician Emmy Noether) on http://petfinder.com.  The website provides an API so that other sites can easily access the data on their databases.  We will also import requests_cache, so that we can cache the results of a request by default.  This is typically a kind thing to do so that if we make the same request then it will not overburden the webserver.

In [2]:
import requests_cache
import pandas as pd
from matplotlib import pyplot as plt

plt.style.use('ggplot')
requests_cache.install_cache('pet_cache')

In [152]:
key = "12345" #get your own key!

Let's get the dog breeds from petfinder.  The breed_uri variable specifies my key (you will need to request your own), the method that I want to use, breed.list, the species, and the format of the output (JSON).  This format is specific to the petfinder api (<a href="https://www.petfinder.com/developers/api-docs">you can look at the documentation</a>).

In [4]:
params = {'key': key, 'animal': 'dog', 'format': 'json'}

We will store the parameters in the above dictionary and the root url in the string below.

In [5]:
breed_url = "http://api.petfinder.com/breed.list"

These are combined to make the request using ``requests.get``.  

In [8]:
req = requests.get(breed_url,params=params)

What this does is make the request to the webserver and pass those parameters.  You can see the url that this constructs with the parameters.

In [156]:
req.url

'http://api.petfinder.com/breed.list?animal=dog&format=json&key=12345'

We can extract the json that the request received.

In [9]:
js = req.json()

print(js.__repr__()[0:300])

{'@encoding': 'iso-8859-1', '@version': '1.0', 'petfinder': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', 'breeds': {'breed': [{'$t': 'Affenpinscher'}, {'$t': 'Afghan Hound'}, {'$t': 'Airedale Terrier'}, {'$t': 'Akbash'}, {'$t': 'Akita'}, {'$t': 'Alaskan Malamute'}, {'$t': 'American Bu


We see that at the top is a dictionary (object in json) and most of the content is in the 'petfinder' key.  Then there is a list (array) of breeds which are dictionaries with the 'dollart' key.  Each API has its own idiosyncracies and this one has that all of the data has the key 'dollart'.

In [25]:
req.status_code

200

We can also check the status code of the request which tell us that it was successful (200 is a successful code as opposed to codes in the 400's).

You can explore the tree structure of the data by looking at the keys and descending.

In [27]:
print(js.keys())

dict_keys(['@encoding', '@version', 'petfinder'])


In [28]:
js['petfinder'].keys()

dict_keys(['@xmlns:xsi', 'breeds', 'header', '@xsi:noNamespaceSchemaLocation'])

In [10]:
js['petfinder']['breeds'].keys()

dict_keys(['breed', '@animal'])

Now we know how to extract the dog breeds.

In [12]:
breeds = [b['$t'] for b in js['petfinder']['breeds']['breed']]

In [13]:
print(", ".join(breeds[:30]))

Affenpinscher, Afghan Hound, Airedale Terrier, Akbash, Akita, Alaskan Malamute, American Bulldog, American Eskimo Dog, American Foxhound, American Hairless Terrier, American Staffordshire Terrier, American Water Spaniel, Anatolian Shepherd, Appenzell Mountain Dog, Australian Cattle Dog / Blue Heeler, Australian Kelpie, Australian Shepherd, Australian Terrier, Basenji, Basset Hound, Beagle, Bearded Collie, Beauceron, Bedlington Terrier, Belgian Shepherd / Laekenois, Belgian Shepherd / Malinois, Belgian Shepherd / Sheepdog, Belgian Shepherd / Tervuren, Bernese Mountain Dog, Bichon Frise


In [14]:
len(breeds)

257

We can see that there are 257 breeds of dogs in their database.  These include for example, the 'Bedlington Terrier'.

The Petfinder API also includes a ``pet.getRandom`` method which allows one to randomly sample the database for a pet with a specific breed.

In [16]:
pet_url_ex = "http://api.petfinder.com/pet.getRandom"
randomparms = {'key':key,'animal':'dog','format':'json','output':'basic'}
randreq = requests.get(pet_url_ex,params = randomparms)
js = randreq.json()

In [18]:
print(js['petfinder'].__repr__()[0:700])

{'pet': {'options': {'option': [{'$t': 'altered'}, {'$t': 'hasShots'}, {'$t': 'housetrained'}]}, 'status': {'$t': 'A'}, 'contact': {'phone': {'$t': '210-535-5480'}, 'state': {'$t': 'TX'}, 'address2': {'$t': 'P.O. Box 743'}, 'email': {'$t': 'teresakopacki@gmail.com'}, 'city': {'$t': 'Lytle'}, 'zip': {'$t': '78052'}, 'fax': {}, 'address1': {'$t': '17971 W. FM 2790 S.'}}, 'age': {'$t': 'Adult'}, 'size': {'$t': 'S'}, 'media': {'photos': {'photo': [{'@size': 'pnt', '$t': 'http://photos.petfinder.com/photos/pets/40700638/1/?bust=1516246159&width=60&-pnt.jpg', '@id': '1'}, {'@size': 'fpm', '$t': 'http://photos.petfinder.com/photos/pets/40700638/1/?bust=1516246159&width=95&-fpm.jpg', '@id': '1'}, {'


We can see some of the information that the getRandom method returns above, including the state, age, size, etc.  By examining this we can get some sense of the pet.  We can for example, print the following,

In [24]:
def print_pet(js):
    """
    Input: Petfinder JSON object from getRandom method
    Output: String describing the pet
    """
    breed_obj = js[u'petfinder'][u'pet'][u'breeds'][u'breed']
    if type(breed_obj) == list:
        breeds = [a[u'$t'] for a in breed_obj]
        breed = ", ".join(breeds) + " mix"
    else:
        breed = breed_obj[u'$t']
    name = js[u'petfinder'][u'pet'][u'name'][u'$t']
    desc = js[u'petfinder'][u'pet'][u'description'][u'$t']
    return "{} is a {}. {}".format(name,breed,desc)

In [40]:
print(print_pet(js))

Murray is a Rat Terrier. Meet MURRAY!!

Murray is an approximately 2 year old, 18-20 pound, neutered, male terrier mix breed. Possibly a Rat Terrier blend.

Murray was originally found as a stray and was in a semi-rural outdoor shelter near Lytle. The ACO reached out to us and we happily brought Murray into our program.

Murray is a gentlemanly little guy. He is very social, but generally minds his own business, and likes to be active. Murray is a chunky guy and needs an active person to help get him fit. He does well with the other dogs in his foster home, but does not like other pushy or bossy males picking on him. He has not shown alot of interest in cats, but as a typical terrier may give chase. He is crate trained and does his business outside when kept on a consistent schedule. Murray is loyal and loves his people.

Murray is heartworm negative, up to date on vaccinations, is current on flea and heartworm preventive, and is micro-chipped. He comes with a health certificate if tra

It turns out that we can also see the image url's for the dog, in the following branch.

In [20]:
image_recs = js[u'petfinder'][u'pet'][u'media'][u'photos'][u'photo']
image_recs[0]

{'@size': 'pnt',
 '$t': 'http://photos.petfinder.com/photos/pets/40700638/1/?bust=1516246159&width=60&-pnt.jpg',
 '@id': '1'}

I will display the pet and description with the following code.

In [21]:
from IPython.display import HTML

In [22]:
def display_pet(js):
    """
    Input: petfinder json object
    Output: html string with image
    """
    try:
        image_recs = js[u'petfinder'][u'pet'][u'media'][u'photos'][u'photo']
    except KeyError:
        return ""
    for rec in image_recs:
        image_uri = rec['$t']
        if rec['@size'] == u'x':
            break
    return "<center><img src='{}'></center>".format(image_uri)

In [25]:
HTML(display_pet(js) + "<pre>" + print_pet(js) + "</pre>") 

Let's streamline this process with the following function.

In [26]:
def random_dog(key):
    """
    Input: api key
    Output: HTML description of the dog
    """
    pet_url_ex = "http://api.petfinder.com/pet.getRandom"
    randomparms = {'key':key,'animal':'dog','format':'json','output':'basic'}
    randreq = requests.get(pet_url_ex,params = randomparms)
    js = randreq.json()
    return js

If we want to make the same request then we will need to disable requests_cache, since it will just find the previous return.

In [27]:
with requests_cache.disabled():
    js = random_dog(key)
HTML(display_pet(js) + "<pre>" + print_pet(js) + "</pre>") 

Now we are ready to get a random sample of the dataset.  This should only be done sparingly because it will make many requests to the webserver in a short period of time.  Many APIs including the petfinder API have rate limits that restrict the number of queries a single key can make.

In [30]:
## Make 500 requests to the petfinder API of random dogs
samp_size = 500

with requests_cache.disabled():
    dog_data = [random_dog(key) for s in range(samp_size)]

In order to help us extract the data and handle missingness, we will make the following helper functions.

In [31]:
def extract_breeds(pet):
    """Extract the breed information for petfinder json"""
    try:
        breed_obj = pet[u'breeds'][u'breed']
        if type(breed_obj) == list:
            breeds = [a[u'$t'] for a in breed_obj]
        else:
            breeds = [breed_obj[u'$t']]
        return breeds
    except KeyError:
        return None

In [32]:
def catch_missing(var_dict,key):
    """Catch missingness in a variable and return None"""
    try:
        var = var_dict[key]['$t']
        return var
    except KeyError:
        return None

In [33]:
def extract_pet_vars(js):
    """Extract the desired variables from petfinder json"""
    pet = js['petfinder']['pet']
    pet_breeds = extract_breeds(pet)
    pet_cont = pet['contact']
    pet_state = catch_missing(pet_cont,'state')
    pet_age = catch_missing(pet,'age')
    pet_size = catch_missing(pet,'size')
    pet_id = int(catch_missing(pet,'id'))
    pet_desc = catch_missing(pet,'description')
    pet_shelterId = catch_missing(pet,'shelterId')
    return {'breeds':pet_breeds, 'state':pet_state, 'age':pet_age, 'size':pet_size, 'id':pet_id, 
           'desc': pet_desc, 'shelter_id': pet_shelterId}

For example, for the following single json below we can see what these variables return.

In [34]:
extract_pet_vars(js)

{'breeds': ['Great Pyrenees'],
 'state': 'NC',
 'age': 'Baby',
 'size': 'L',
 'id': 43203450,
 'desc': 'This is Snowman he is a 10 month old Great  Pyrenees that now needs a new home through no fault of his. He is house trained current on shots and still does dumb puppy things. Snowman needs a home that will continue his training and be able to provide lots of hugs and belly rubs.',
 'shelter_id': 'NC820'}

All variables are strings other than ID (integer) and breeds (list).  We will come back to breeds.  First, let's create a dataframe and set the dog id as the index.

In [35]:
dog_df = pd.DataFrame(extract_pet_vars(js) for js in dog_data)

dog_df = dog_df.set_index('id')

We can describe these variables and find that the most common age is 'adult', and the most common state is Texas.  Not all dogs have a description.

In [36]:
dog_df.describe()

,age,breeds,desc,shelter_id,size,state
count,500,500,447,500,500,500
unique,4,215,438,440,4,49
top,Adult,[Pit Bull Terrier],No Notes,IL192,M,TX
freq,236,51,4,4,245,74


It seems that a common adult dog is a Pit Bull, and a common baby is a Labrador Retriever.

In [37]:
dog_df.groupby('age').describe()

breeds                                    desc         \
        count unique                   top freq count unique   
age                                                            
Adult     236    113    [Pit Bull Terrier]   28   213    208   
Baby       80     60  [Labrador Retriever]    8    73     73   
Senior     48     32           [Chihuahua]    9    45     45   
Young     136     79    [Pit Bull Terrier]   15   116    114   

                                                               shelter_id  \
                                                      top freq      count   
age                                                                         
Adult                                            No Notes    4        236   
Baby    Meet Cosmo! Cosmo is a 6 month old Beagle/Houn...    1         80   
Senior  HIGHLY ADOPTABLE!!! ROSIE is a Jack Russell Te...    1         48   
Young   Craigie D Boss came to us from a kill shelter ...    3        136   

                           size                 state                  
       unique    top freq count unique top freq count unique top freq  
age                                                                    
Adult     223   TN75    2   236      4   M  100   236     43  TX   31  
Baby       77  MN289    2    80      4   M   47    80     31  TX   10  
Senior     45  NV200    2    48      4   S   22    48     23  CA    7  
Young     129  IN434    3   136      3   M   84   136     31  TX   27

Each breed is a list of breeds (for when the dog is mixed).  We can see which breeds are the most common when we consider a dog to possibly count for multiple breeds at once.

In [109]:
from collections import Counter

all_breeds = Counter(sum(dog_df['breeds'].values,[]))

In [113]:
all_breeds.most_common()[:5]

[('Labrador Retriever', 89),
 ('Pit Bull Terrier', 76),
 ('Mixed Breed', 67),
 ('Chihuahua', 37),
 ('Shepherd', 26)]

Labrador Retriever and Pit Bull are the most numerous.

It is informative to check what the maximum number of breeds are, which turns out to be two.  This is likely imposed by when the users enter in a dog's information.

In [116]:
max(len(br) for br in dog_df['breeds'].values)

2

Let's encode these breeds as variables breed1 and breed2, and if there is only one breed then these will have the same value.  

In [117]:
dog_df['breed1'] = [br[0] for br in dog_df['breeds'].values]
dog_df['breed2'] = [br[-1] for br in dog_df['breeds'].values]

Now we can make more detailed queries, such as for a pure-bred Lab.

In [126]:
lab_dogs = dog_df.query('breed1 == "{0}" or breed2 == "{0}"'.format("Labrador Retriever"))
lab_dog = lab_dogs.iloc[0,:]

In [127]:
lab_dog

age                                                       Adult
breeds                                     [Labrador Retriever]
desc          This is the Animal Description Header\nRover i...
shelter_id                                                CA387
size                                                          L
state                                                        CA
breed1                                       Labrador Retriever
breed2                                       Labrador Retriever
Name: 42874092, dtype: object

In [129]:
print(lab_dog['desc'])

This is the Animal Description Header
Rover is all dog, hence the name! He loves people and is very confident and playful in his surroundings. He will need some training since he can be mouthy in play and is jumpy with excitement when meeting new people. Rover is a hunting stock kind of Lab mix, lanky and energetic - he needs space and adventures!; This is a high energy boy who will need and love lots of exercise. He will make an excellent dog for outdoorsy folks who have the time and sense of adventure to go on long hikes, to do some training and recall work - he'll likely be really good and he does want to please - but he needs an outlet for his enthusiasm!This is the Animal Description Footer
10/23/18 2:49 AM


This process allows us to prototype the dogs, by selecting some examples and looking at their data.

Let's also look at the most common states.

In [132]:
dog_df.groupby('state').count().sort_values('age',ascending=False).iloc[:5,0]

state
TX    75
CA    55
FL    30
GA    28
OH    19
Name: age, dtype: int64

Let's look for a Pitt Bull in Texas,

In [136]:
pitt_dog = dog_df.query('state == "TX" and breed1 == "Pit Bull Terrier"').iloc[0,:]

In [137]:
pitt_dog

age                                                       Adult
breeds                                       [Pit Bull Terrier]
desc          I'm a petite little lady who loves to give kis...
shelter_id                                                TX198
size                                                          M
state                                                        TX
breed1                                         Pit Bull Terrier
breed2                                         Pit Bull Terrier
Name: 41596889, dtype: object

In [139]:
pitt_dog['desc']

"I'm a petite little lady who loves to give kisses! I'm sweet and enjoy lots of belly rubs, and I'm also a pretty amazing athlete! I like playing with toys, going for runs, and sleeping next you once I'm all tuckered out. I would prefer to have all your attention to myself, so I'd do best an only pet - I'm such a good girl, I'm the only companion you need! Primary Color: Black Secondary Color: White Weight: 43.4lbs Age: 3yrs 11mths 1wks Animal has been Spayed"